# Simple generator with STARlight-like format

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Event generation parameters
n_photon_photon = 100000
n_jpsi = 10000
n_psi2S_muons = 200
n_psi2S_jpsi_pions = 100

# Constants (GeV/c^2)
m_muon = 0.105658  # Muon mass
m_jpsi = 3.0969    # J/psi mass
m_psi2S = 3.6861   # Psi(2S) mass
m_pion = 0.13957   # Charged pion mass

def isotropic_two_body_decay(mother_mass, daughter_mass1, daughter_mass2):
    E1 = (mother_mass**2 + daughter_mass1**2 - daughter_mass2**2) / (2 * mother_mass)
    E2 = (mother_mass**2 + daughter_mass2**2 - daughter_mass1**2) / (2 * mother_mass)
    p = np.sqrt(E1**2 - daughter_mass1**2)
    p = tpc_momentum_resolution(p)
    theta = np.arccos(2 * np.random.rand() - 1)
    phi = 2 * np.pi * np.random.rand()
    p1 = np.array([p * np.sin(theta) * np.cos(phi), p * np.sin(theta) * np.sin(phi), p * np.cos(theta)])
    p2 = -p1
    return p1, p2, E1, E2

def tpc_momentum_resolution(momentum):
    a = 0.005
    b = 0.01
    resolution = np.sqrt(a**2 + (b * momentum)**2)
    smeared_momentum = momentum * (1 + np.random.normal(0, resolution))
    return smeared_momentum

def generate_jpsi_events(n_events):
    events = []
    for _ in range(n_events):
        smearedTracks = []
        muon1_p, muon2_p, E1, E2 = isotropic_two_body_decay(m_jpsi, m_muon, m_muon)
        # muon1_p = tpc_momentum_resolution(np.linalg.norm(muon1_p))
        # muon2_p = tpc_momentum_resolution(np.linalg.norm(muon2_p))
        smearedTracks.append(muon1_p)
        smearedTracks.append(muon2_p)
        events.append(smearedTracks)
        # events.append(mass)
    return np.array(events)


def format_event_as_starlight(event, event_number, mother_mass, daughter_mass, mother_momentum, mother_energy, pdg_mother, pdg_daughter):
    """
    Formats a single event into STARlight-like string format.
    
    Parameters:
    - event: List of particle tracks (3D momenta) in the event.
    - event_number: The event number (starting from 1).
    - mother_mass: Mass of the mother particle.
    - daughter_mass: Mass of the daughter particle.
    - mother_momentum: 3D momentum (px, py, pz) of the mother particle.
    - mother_energy: Energy of the mother particle.
    - pdg_mother: PDG code of the mother particle.
    - pdg_daughter: PDG code of the daughter particle.
    
    Returns:
    - event_string: A string representing the event in STARlight format
    """
    # Initialize event string
    event_string = f"EVENT: {event_number} {len(event)} 1\n"
    
    # Add vertex information (assuming x, y, z, t are all 0, and 1 process, 0 parent, 2 daughters)
    event_string += "VERTEX: 0 0 0 0 1 0 0 2\n"
    
    # Mother particle (assuming mother particle stops at vertex 0)
    px_m, py_m, pz_m = mother_momentum
    # event_string += f"TRACK: {pdg_mother} {px_m:.6f} {py_m:.6f} {pz_m:.6f} {event_number} 0 0 {pdg_mother}\n"
    
    # Add daughter tracks
    for track_number, track in enumerate(event):
        px, py, pz = track[:3]
        energy = np.sqrt(np.linalg.norm(track[:3])**2 + daughter_mass**2)  # Calculate energy from momentum and daughter mass
        event_string += f"TRACK: {pdg_daughter} {px:.6f} {py:.6f} {pz:.6f} {event_number} {track_number + 1} 0 {pdg_daughter}\n"
    
    return event_string


def convert_events_to_starlight_format(events, mother_mass, daughter_mass, mother_momenta, mother_energies, pdg_mother, pdg_daughter):
    """
    Converts a list of events into STARlight-like string format.
    
    Parameters:
    - events: List of events, where each event is a list of tracks.
              Each track contains [px, py, pz].
    - mother_mass: Mass of the mother particle.
    - daughter_mass: Mass of the daughter particle.
    - mother_momenta: List of 3D momentum vectors for the mother particles.
    - mother_energies: List of energies for the mother particles.
    - pdg_mother: PDG code of the mother particle.
    - pdg_daughter: PDG code of the daughter particle.
    
    Returns:
    - starlight_output: A string representing all events in STARlight format
    """
    starlight_output = ""
    for event_number, (event, mother_momentum, mother_energy) in enumerate(zip(events, mother_momenta, mother_energies), start=1):
        starlight_output += format_event_as_starlight(event, event_number, mother_mass, daughter_mass, mother_momentum, mother_energy, pdg_mother, pdg_daughter)
        # starlight_output += "\n"  # Separate each event by a newline
    return starlight_output


# Example usage:
n_jpsi = 1000
events_jpsi = generate_jpsi_events(n_jpsi)

# Masses of the mother and daughter particles (J/psi and muons in this case)
mother_mass = m_jpsi  # J/psi mass
daughter_mass = m_muon  # Muon mass

# PDG codes for J/psi and muons
pdg_mother = 443  # PDG code for J/psi
pdg_daughter = 13  # PDG code for muons

# For demonstration purposes, let's assume J/psi has 0 initial momentum and an energy equal to its mass (rest energy)
mother_momenta_jpsi = [[0.0, 0.0, 0.0] for _ in range(n_jpsi)]
mother_energies_jpsi = [mother_mass for _ in range(n_jpsi)]

# Convert events to STARlight-like output
starlight_output_jpsi = convert_events_to_starlight_format(
    events_jpsi, 
    mother_mass, 
    daughter_mass, 
    mother_momenta_jpsi, 
    mother_energies_jpsi, 
    pdg_mother, 
    pdg_daughter
)

print(starlight_output_jpsi)


EVENT: 1 2 1
VERTEX: 0 0 0 0 1 0 0 2
TRACK: 13 1.450219 -0.474372 0.233393 1 1 0 13
TRACK: 13 -1.450219 0.474372 -0.233393 1 2 0 13
EVENT: 2 2 1
VERTEX: 0 0 0 0 1 0 0 2
TRACK: 13 0.847326 0.823473 -0.985523 2 1 0 13
TRACK: 13 -0.847326 -0.823473 0.985523 2 2 0 13
EVENT: 3 2 1
VERTEX: 0 0 0 0 1 0 0 2
TRACK: 13 1.511554 0.049581 -0.385990 3 1 0 13
TRACK: 13 -1.511554 -0.049581 0.385990 3 2 0 13
EVENT: 4 2 1
VERTEX: 0 0 0 0 1 0 0 2
TRACK: 13 0.103943 1.481450 -0.574811 4 1 0 13
TRACK: 13 -0.103943 -1.481450 0.574811 4 2 0 13
EVENT: 5 2 1
VERTEX: 0 0 0 0 1 0 0 2
TRACK: 13 -0.139928 1.514765 -0.130679 5 1 0 13
TRACK: 13 0.139928 -1.514765 0.130679 5 2 0 13
EVENT: 6 2 1
VERTEX: 0 0 0 0 1 0 0 2
TRACK: 13 1.088237 0.333765 -1.054941 6 1 0 13
TRACK: 13 -1.088237 -0.333765 1.054941 6 2 0 13
EVENT: 7 2 1
VERTEX: 0 0 0 0 1 0 0 2
TRACK: 13 -0.513275 -0.339839 1.396483 7 1 0 13
TRACK: 13 0.513275 0.339839 -1.396483 7 2 0 13
EVENT: 8 2 1
VERTEX: 0 0 0 0 1 0 0 2
TRACK: 13 0.650655 -1.422059 0.228193 8